## Data Preparation for Hedonic Regression Modelling 

The following analysis includes an examination of multi-colinearity between the variables for the study, which is an important step in the preparation of data for linear hedonic modelling. Multicollinearity can make it difficult to interpret regression coefficients, as it causes large standard errors, which can result in insignificant predictors even when they are important.
It doesn't affect the predictive power of the model but complicates understanding the relationship between predictors and the response variable. Multicollinearity occurs when two or more predictor variables in a regression model are highly correlated, which can make it difficult to determine the individual contribution of each predictor to the dependent variable.

We also transform variables so the scale of variables doesn't impact the coefficients.

In [1]:
# Read in libraries 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import MinMaxScaler
import geopandas as gpd

In [2]:
# Read in the data 
df = pd.read_csv('cleaned_data_hp_aircon.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578080 entries, 0 to 578079
Data columns (total 60 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0_left                578080 non-null  int64  
 1   ASSETID                        578080 non-null  object 
 2   PERIOD                         578080 non-null  int64  
 3   PRICE                          578080 non-null  float64
 4   UNITPRICE                      578080 non-null  float64
 5   CONSTRUCTEDAREA                578080 non-null  int64  
 6   ROOMNUMBER                     578080 non-null  int64  
 7   BATHNUMBER                     578080 non-null  int64  
 8   HASTERRACE                     578080 non-null  int64  
 9   HASLIFT                        578080 non-null  int64  
 10  HASAIRCONDITIONING             578080 non-null  int64  
 11  AMENITYID                      578080 non-null  int64  
 12  HASPARKINGSPACE               

### Calculate % of listings and neighbourhoods with Air Conditioning 

In [107]:
# Calculate the overall percentage of listings with air conditioning
overall_aircon_percentage = df['HASAIRCONDITIONING'].mean() * 100

# Print the result
print(f"Overall percentage of listings with air conditioning: {overall_aircon_percentage:.2f}%")


Overall percentage of listings with air conditioning: 36.54%


In [95]:
# Group by 'COD_BAR' and calculate the percentage of listings with air conditioning
airconditioning_percentage = (
    df.groupby('CODBAR')['HASAIRCONDITIONING']
    .mean()  # This calculates the mean, which corresponds to the percentage for binary variables
    .mul(100)  # Convert to percentage
    .reset_index(name='%_With_AirConditioning')  # Reset index and name the column
)


In [85]:
# Read in neighbourhood shapefile
barrios = gpd.read_file('barrios.shp')

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/envs/gds/share/proj failed


In [89]:
# Rename columns to match
barrios = barrios.rename(columns={'COD_BAR': 'CODBAR'})

In [100]:
# Change column type to match
barrios['CODBAR'] = barrios['CODBAR'].astype(int)

In [101]:
# Merge the dfs
merged_df = pd.merge(airconditioning_percentage, barrios, on='CODBAR', how='outer')

In [106]:
# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(merged_df, geometry=merged_df['geometry'])

# Set the coordinate reference system (CRS) - you can set this to a specific CRS, e.g., 'EPSG:4326' for WGS84
gdf.set_crs(epsg=25830, inplace=True)  # You can replace 4326 with another CRS if necessary

# Save the GeoDataFrame as a shapefile
gdf.to_file("neighbourhood_aircon.shp")

/tmp/ipykernel_6195/3868839955.py:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("neighbourhood_aircon.shp")


## Calculate Correlation Coefficients Between Predictors

* 1: A perfect positive correlation. As one variable increases, the other also increases in a proportional manner.
* 0: No correlation. Changes in one variable do not predict changes in the other.
* -1: A perfect negative correlation. As one variable increases, the other decreases proportionally.

In [198]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578080 entries, 0 to 578079
Data columns (total 60 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0_left                578080 non-null  int64  
 1   ASSETID                        578080 non-null  object 
 2   PERIOD                         578080 non-null  int64  
 3   PRICE                          578080 non-null  float64
 4   UNITPRICE                      578080 non-null  float64
 5   CONSTRUCTEDAREA                578080 non-null  int64  
 6   ROOMNUMBER                     578080 non-null  int64  
 7   BATHNUMBER                     578080 non-null  int64  
 8   HASTERRACE                     578080 non-null  int64  
 9   HASLIFT                        578080 non-null  int64  
 10  HASAIRCONDITIONING             578080 non-null  int64  
 11  AMENITYID                      578080 non-null  int64  
 12  HASPARKINGSPACE               

In [201]:
# List of columns to drop that you dont want to calculate correlation matrix for
columns_to_drop = ['UNITPRICE', 'Log_UNITPRICE', 'code_dis', 'CODBAR', 'geometry', 'ASSETID','PRICE','LONGITUDE', 'LATITUDE',
    'NOMBRE_x','area_x','index_right','name_dis', 'pop_2021', 'Unnamed: 0_right', 'Unnamed: 0_left']

# Drop the columns from the DataFrame
df2 = df.drop(columns=columns_to_drop, axis=1)

In [202]:
# Calculate correlation matrix
correlation_matrix = df2.corr()

In [203]:
# Flatten and sort the correlation matrix
def get_highest_correlations(corr_matrix, n=5):
    # Reset the upper triangle values (mask them)
    
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    corr_matrix = corr_matrix.mask(mask)
    
    # Unstack the matrix into a Series and drop NaNs
    sorted_correlations = corr_matrix.unstack().dropna()
      
    # Sort absolute correlations in descending order
    sorted_correlations = sorted_correlations.abs().sort_values(ascending=False)
    
    # Return the top N correlations
    return sorted_correlations.head(n)

# Example: Get the top 10 highest correlations
top_correlations = get_highest_correlations(correlation_matrix, n=15)

print(top_correlations)


HASPARKINGSPACE      ISPARKINGSPACEINCLUDEDINPRICE    1.000000
%_over65_2021        Average_age_2021                 0.951662
%_singlehouse_2021   avg_house_size_2021              0.934985
avg_house_size_2021  %_u18_2021                       0.812827
%_singlehouse_2021   %_u18_2021                       0.756553
%_u18_2021           Average_age_2021                 0.744344
building_density     pop_den                          0.694653
PERIOD               AMENITYID                        0.682168
%_singlehouse_2021   pop_den                          0.679537
HASSWIMMINGPOOL      HASGARDEN                        0.665221
avg_house_size_2021  pop_den                          0.655577
%_spanish_2021       building_density                 0.647613
%_singlehouse_2021   building_density                 0.636455
%_u18_2021           pop_den                          0.627579
income_2019          green_area_percent               0.604529
dtype: float64


## Variables to drop based on Correlation Coefficient 
* IS PARKING SPACE INCLUDED IN PRICE
* Average house size
* Percentage single house
* Population Density
* Percentage Spanish
* Average age 2021
  
These variables are removed from the final variable list as they have the highest correlation coefficients. None of the continous variables now have correlation coefficients over 0.61.

In [204]:
# List of columns to drop
columns_to_drop = ['ISPARKINGSPACEINCLUDEDINPRICE', 'avg_house_size_2021', '%_singlehouse_2021', 'Average_age_2021',
                   'pop_den', '%_spanish_2021']


# Drop the columns from the DataFrame
df_new = df.drop(columns=columns_to_drop, axis=1)

In [205]:
# Drop the columns from the DataFrame
df2 = df2.drop(columns=columns_to_drop, axis=1)

### Calculate the VIF

Variance Inflation Factor (VIF) is a measure used to detect multicollinearity in regression analysis. VIF quantifies how much the variance of a regression coefficient is inflated due to collinearity with other predictors. 

* VIF = 1: No correlation between the predictor and other predictors (ideal).
* 1 < VIF ≤ 5: Moderate correlation, not typically problematic.
* VIF > 5: Possible multicollinearity concern; further investigation needed.
* VIF > 15: High multicollinearity; problematic and requires action (e.g., removing or combining variables)


In [206]:

# Calculate VIF for each predictor
vif = pd.DataFrame()
vif['Variable'] = df2.columns
vif['VIF'] = [variance_inflation_factor(df2.values, i) for i in range(df2.shape[1])]


In [207]:
# Filter and print variables with VIF greater than 10
vif_high = vif[vif['VIF'] > 10]

# Sort by VIF value in descending order
vif_high_sorted = vif_high.sort_values(by='VIF', ascending=False)

# Print the sorted dataframe
print(vif_high_sorted)

                 Variable          VIF
25    CADCONSTRUCTIONYEAR  9619.991846
0                  PERIOD  9507.681630
30             %_u18_2021    53.539666
31          %_over65_2021    43.547082
23            BUILTTYPEID    38.230378
29            income_2019    35.859231
28     CADASTRALQUALITYID    24.068505
35           tree_density    21.152858
34     green_area_percent    16.166416
24         FLATLOCATIONID    16.119134
37  CoolingDemandCategory    14.413909
2              ROOMNUMBER    12.639129
7               AMENITYID    12.059282
3              BATHNUMBER    10.949534


Based on the high VIF values, we remove:
* CADCONSTRUCTIONYEAR
* PERIOD
* Tree Density
* Percentage_u18_2021
* BUILTTYPEID
* CADASTRALQUALITYID
* FLATLOCATIONID
* AMENITYID
* BATHNUMBER
* Percentage over 65


In [208]:
# List of columns to drop
columns_to_drop2 = ['CADCONSTRUCTIONYEAR', 'PERIOD', 'tree_density',
    '%_u18_2021', 'BUILTTYPEID', 'CADASTRALQUALITYID', 
    'FLATLOCATIONID', 'AMENITYID', 'BATHNUMBER', '%_over65_2021'
]

# Drop the columns from the DataFrame
df_new = df_new.drop(columns=columns_to_drop2, axis=1)


In [209]:
# Drop the columns from the DataFrame
df3 = df2.drop(columns=columns_to_drop2, axis=1)

In [117]:
# Calculate VIF for each predictor
vif = pd.DataFrame()
vif['Variable'] = df3.columns
vif['VIF'] = [variance_inflation_factor(df3.values, i) for i in range(df3.shape[1])]

In [118]:
# Filter and print variables with VIF greater than 10
vif_high = vif[vif['VIF'] > 10]

# Sort by VIF value in descending order
vif_high_sorted = vif_high.sort_values(by='VIF', ascending=False)

# Print the sorted dataframe
print(vif_high_sorted)

                 Variable        VIF
22            income_2019  12.075446
27  CoolingDemandCategory  10.293541


### Final Dataframe 

In [58]:
# Adjust the display settings
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # Adjust the width to show all column

In [130]:
## Make table of summary statistics in the raw form
df3.describe().T

,count,mean,std,min,25%,50%,75%,max
CONSTRUCTEDAREA,578090.0,106.113287,887.906970,1.000000,64.000000,84.000000,117.000000,3.990000e+05
ROOMNUMBER,578090.0,2.650136,1.170535,0.000000,2.000000,3.000000,3.000000,1.500000e+01
HASTERRACE,578090.0,0.323870,0.467951,0.000000,0.000000,0.000000,1.000000,1.000000e+00
HASLIFT,578090.0,0.695544,0.460177,0.000000,0.000000,1.000000,1.000000,1.000000e+00
HASAIRCONDITIONING,578090.0,0.365365,0.481533,0.000000,0.000000,0.000000,1.000000,1.000000e+00
HASPARKINGSPACE,578090.0,0.225314,0.417790,0.000000,0.000000,0.000000,0.000000,1.000000e+00
PARKINGSPACEPRICE,578090.0,411.222313,5609.620774,1.000000,1.000000,1.000000,1.000000,1.200001e+06
HASNORTHORIENTATION,578090.0,0.085975,0.280327,0.000000,0.000000,0.000000,0.000000,1.000000e+00
HASSOUTHORIENTATION,578090.0,0.201647,0.401230,0.000000,0.000000,0.000000,0.000000,1.000000e+00
HASEASTORIENTATION,578090.0,0.171378,0.376839,0.000000,0.000000,0.000000,0.000000,1.000000e+00


In [131]:
## Make table of summary statistics in the raw form
df.UNITPRICE.describe().T

count    578090.000000
mean       2990.539671
std        1379.852830
min           0.974937
25%        1900.000000
50%        2795.698962
75%        3888.889000
max        7090.909091
Name: UNITPRICE, dtype: float64

## Normalising Variables

In [210]:
scaler = MinMaxScaler()
df_new[['CONSTRUCTEDAREA', 'green_area_percent','CoolingDemandCategory','ROOMNUMBER','building_density', 'bench_density', 'FLOORCLEAN', 'CADMAXBUILDINGFLOOR', 'CADDWELLINGCOUNT', 'PARKINGSPACEPRICE']] = scaler.fit_transform(
    df_new[['CONSTRUCTEDAREA', 'green_area_percent','CoolingDemandCategory','ROOMNUMBER','building_density', 'bench_density', 'FLOORCLEAN', 'CADMAXBUILDINGFLOOR', 'CADDWELLINGCOUNT', 'PARKINGSPACEPRICE']]
)

In [211]:
df_new['log_income'] = np.log(df_new['income_2019'])

In [213]:
# List of columns to drop
columns_to_drop3 = ['UNITPRICE','LONGITUDE','LATITUDE', 'PRICE', 'income_2019',
    'NOMBRE_x','area_x','index_right','name_dis', 'pop_2021','Unnamed: 0_left', 'ASSETID', 'Unnamed: 0_right']

In [214]:
# Drop the columns from the DataFrame
df_new2 = df_new.drop(columns=columns_to_drop3, axis=1)

In [215]:
df_new2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578080 entries, 0 to 578079
Data columns (total 32 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CONSTRUCTEDAREA        578080 non-null  float64
 1   ROOMNUMBER             578080 non-null  float64
 2   HASTERRACE             578080 non-null  int64  
 3   HASLIFT                578080 non-null  int64  
 4   HASAIRCONDITIONING     578080 non-null  int64  
 5   HASPARKINGSPACE        578080 non-null  int64  
 6   PARKINGSPACEPRICE      578080 non-null  float64
 7   HASNORTHORIENTATION    578080 non-null  int64  
 8   HASSOUTHORIENTATION    578080 non-null  int64  
 9   HASEASTORIENTATION     578080 non-null  int64  
 10  HASWESTORIENTATION     578080 non-null  int64  
 11  HASBOXROOM             578080 non-null  int64  
 12  HASWARDROBE            578080 non-null  int64  
 13  HASSWIMMINGPOOL        578080 non-null  int64  
 14  HASDOORMAN             578080 non-nu

In [216]:
# Rename the column '2000_18' to 'sat_image_cluster'
df_new2.rename(columns={'2000_18': 'sat_image_clus'}, inplace=True)


In [217]:
# Save df_new2 to a CSV file
df_new2.to_csv('df_new2finalmodeldata.csv', index=False)